# TDT05 Machine Learning in practice challenge 2
***Victor Jørgensen and Hans Kristian Sande***

This notebook is heavily inspired by this tutorial: https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

## Import modules

In [21]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import catboost
from catboost import CatBoostClassifier, Pool, metrics, cv
import graphviz
import pandas as pd
import numpy as np
print("No errors, all good!")

No errors, all good!


## 1.1 Data loading
Load data from csv files using pandas.

In [3]:
df_test = pd.read_csv("data/challenge2_test.csv")
# Uncomment line below to preview five first rows
#df_test.head()

df_train = pd.read_csv("data/challenge2_train.csv")
# Uncomment line below to preview five first rows
df_train.head()

,id,target,f0,f1,f2,f3,f4,f5,f6,f7,...,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28
0,0,0,1.0,gL,e,3.0,A,NaN,0.0,6.0,...,0.5,0.0,3.0,R,328b0cf4e,0.834041,T,N,1.0,14.2364
1,1,0,0.0,Rj,c,1.0,A,7.0,1.0,4.0,...,0.4,0.0,1.0,NaN,328b0cf4e,0.686021,T,N,1.0,NaN
2,2,0,NaN,In,a,1.0,A,10.0,1.0,6.0,...,0.5,1.0,3.0,G,0c67fcbbd,1.141271,T,N,3.0,NaN
3,3,1,1.0,rA,c,3.0,A,7.0,1.0,1.0,...,0.6,1.0,1.0,G,fee4e3007,0.662382,T,N,3.0,NaN
4,4,0,1.0,pE,c,3.0,A,7.0,0.0,6.0,...,0.5,0.0,1.0,B,587e040bd,-1.000000,T,N,1.0,13.9537


## 1.2 Feature preparation
First of all let's check how many absent values do we have:

In [4]:
null_value_stats = df_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

f0      1459
f1      1487
f2      1439
f3      1488
f4      1498
f5     11617
f6      1490
f7      1525
f8      1490
f9      1489
f10     1501
f11      145
f12     1541
f13     1447
f14     1451
f15     1477
f16     1460
f17     9762
f18     1556
f19     1437
f20     1464
f21     1510
f22     1513
f23     1516
f24     2696
f25     1537
f26     1474
f27     1464
f28    13112
dtype: int64

As we se, null values range from ~3% to 15%. Almost all features have a substantial amount of null values, so let's fill them with some number way out of their distributions - so the model would be able to easily distinguish between them and take it into account:

In [5]:
df_train.fillna(-999, inplace=True)
df_test.fillna(-999, inplace=True)

Next we want to separate **feature** and **label** into **X** and **y**.

In [6]:
X = df_train.drop('target', axis=1)
y = df_train.target

Our features are of different data types. Some are numeric, others categorical and other strings. CatBoost allows us to treat these string features just as categorical ones.

In [7]:
X.dtypes

categorical_features_indices = np.where(X.dtypes != float)[0]
categorical_features_indices

array([ 0,  2,  3,  5,  9, 10, 11, 13, 14, 15, 16, 19, 23, 24, 26, 27])

## 1.3 Data splitting
Let's split the train data into training and validation sets

In [17]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = df_test

## 2.1 Model training
Create model. Use default parameters, because they provide a good baseline almost all the time. The only thing we would like to specify here is custom_loss parameter, as this would give us an ability to see what's going on in terms of this competition metric - accuracy, as well as to be able to watch for logloss, as it would be more smooth on dataset of such size.

In [64]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    depth=3,
    iterations=5000,
    logging_level='Silent',
    use_best_model=True
)

In [65]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.038516
0:	learn: 0.6736859	test: 0.6741857	best: 0.6741857 (0)	total: 40.9ms	remaining: 3m 24s
1:	learn: 0.6554786	test: 0.6562861	best: 0.6562861 (1)	total: 60.1ms	remaining: 2m 30s
2:	learn: 0.6381885	test: 0.6391714	best: 0.6391714 (2)	total: 81.3ms	remaining: 2m 15s
3:	learn: 0.6225587	test: 0.6236200	best: 0.6236200 (3)	total: 124ms	remaining: 2m 34s
4:	learn: 0.6090126	test: 0.6104652	best: 0.6104652 (4)	total: 146ms	remaining: 2m 26s
5:	learn: 0.5972520	test: 0.5987191	best: 0.5987191 (5)	total: 177ms	remaining: 2m 27s
6:	learn: 0.5850905	test: 0.5868095	best: 0.5868095 (6)	total: 198ms	remaining: 2m 21s
7:	learn: 0.5743401	test: 0.5760876	best: 0.5760876 (7)	total: 218ms	remaining: 2m 15s
8:	learn: 0.5646781	test: 0.5666042	best: 0.5666042 (8)	total: 245ms	remaining: 2m 15s
9:	learn: 0.5552548	test: 0.5573456	best: 0.5573456 (9)	total: 272ms	remaining: 2m 15s
10:	learn: 0.5463024	test: 0.5486089	best: 0.5486089 (10)	total: 301ms	remaining: 2m 16s
11:	lear

94:	learn: 0.4356029	test: 0.4343145	best: 0.4343145 (94)	total: 3.23s	remaining: 2m 46s
95:	learn: 0.4354852	test: 0.4341590	best: 0.4341590 (95)	total: 3.26s	remaining: 2m 46s
96:	learn: 0.4353217	test: 0.4339990	best: 0.4339990 (96)	total: 3.3s	remaining: 2m 46s
97:	learn: 0.4351476	test: 0.4337855	best: 0.4337855 (97)	total: 3.33s	remaining: 2m 46s
98:	learn: 0.4350093	test: 0.4336115	best: 0.4336115 (98)	total: 3.37s	remaining: 2m 46s
99:	learn: 0.4348486	test: 0.4334272	best: 0.4334272 (99)	total: 3.39s	remaining: 2m 46s
100:	learn: 0.4346840	test: 0.4332643	best: 0.4332643 (100)	total: 3.43s	remaining: 2m 46s
101:	learn: 0.4345361	test: 0.4330813	best: 0.4330813 (101)	total: 3.47s	remaining: 2m 46s
102:	learn: 0.4343988	test: 0.4329326	best: 0.4329326 (102)	total: 3.52s	remaining: 2m 47s
103:	learn: 0.4342624	test: 0.4327724	best: 0.4327724 (103)	total: 3.54s	remaining: 2m 46s
104:	learn: 0.4341211	test: 0.4326075	best: 0.4326075 (104)	total: 3.58s	remaining: 2m 46s
105:	learn: 

187:	learn: 0.4257741	test: 0.4229616	best: 0.4229616 (187)	total: 6.53s	remaining: 2m 47s
188:	learn: 0.4257035	test: 0.4228903	best: 0.4228903 (188)	total: 6.57s	remaining: 2m 47s
189:	learn: 0.4256429	test: 0.4228146	best: 0.4228146 (189)	total: 6.6s	remaining: 2m 47s
190:	learn: 0.4255811	test: 0.4227590	best: 0.4227590 (190)	total: 6.64s	remaining: 2m 47s
191:	learn: 0.4255105	test: 0.4226873	best: 0.4226873 (191)	total: 6.67s	remaining: 2m 47s
192:	learn: 0.4254562	test: 0.4225925	best: 0.4225925 (192)	total: 6.71s	remaining: 2m 47s
193:	learn: 0.4254021	test: 0.4225564	best: 0.4225564 (193)	total: 6.75s	remaining: 2m 47s
194:	learn: 0.4253501	test: 0.4224695	best: 0.4224695 (194)	total: 6.79s	remaining: 2m 47s
195:	learn: 0.4252851	test: 0.4223844	best: 0.4223844 (195)	total: 6.84s	remaining: 2m 47s
196:	learn: 0.4252167	test: 0.4222961	best: 0.4222961 (196)	total: 6.87s	remaining: 2m 47s
197:	learn: 0.4251526	test: 0.4222252	best: 0.4222252 (197)	total: 6.91s	remaining: 2m 47s


280:	learn: 0.4207686	test: 0.4176643	best: 0.4176643 (280)	total: 9.78s	remaining: 2m 44s
281:	learn: 0.4207161	test: 0.4175860	best: 0.4175860 (281)	total: 9.81s	remaining: 2m 44s
282:	learn: 0.4206606	test: 0.4175347	best: 0.4175347 (282)	total: 9.84s	remaining: 2m 44s
283:	learn: 0.4206034	test: 0.4174771	best: 0.4174771 (283)	total: 9.89s	remaining: 2m 44s
284:	learn: 0.4205492	test: 0.4174109	best: 0.4174109 (284)	total: 9.94s	remaining: 2m 44s
285:	learn: 0.4205090	test: 0.4173611	best: 0.4173611 (285)	total: 9.97s	remaining: 2m 44s
286:	learn: 0.4204711	test: 0.4173085	best: 0.4173085 (286)	total: 10s	remaining: 2m 44s
287:	learn: 0.4204309	test: 0.4172671	best: 0.4172671 (287)	total: 10.1s	remaining: 2m 44s
288:	learn: 0.4204009	test: 0.4172223	best: 0.4172223 (288)	total: 10.1s	remaining: 2m 44s
289:	learn: 0.4203581	test: 0.4171901	best: 0.4171901 (289)	total: 10.1s	remaining: 2m 44s
290:	learn: 0.4203337	test: 0.4171661	best: 0.4171661 (290)	total: 10.2s	remaining: 2m 44s
2

...

4567:	learn: 0.3853248	test: 0.4085261	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19.3s
4568:	learn: 0.3853177	test: 0.4085265	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19.2s
4569:	learn: 0.3853148	test: 0.4085288	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19.2s
4570:	learn: 0.3853118	test: 0.4085297	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19.1s
4571:	learn: 0.3853052	test: 0.4085322	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19.1s
4572:	learn: 0.3852870	test: 0.4085377	best: 0.4074796 (2369)	total: 3m 23s	remaining: 19s
4573:	learn: 0.3852771	test: 0.4085403	best: 0.4074796 (2369)	total: 3m 24s	remaining: 19s
4574:	learn: 0.3852737	test: 0.4085352	best: 0.4074796 (2369)	total: 3m 24s	remaining: 19s
4575:	learn: 0.3852707	test: 0.4085352	best: 0.4074796 (2369)	total: 3m 24s	remaining: 18.9s
4576:	learn: 0.3852662	test: 0.4085338	best: 0.4074796 (2369)	total: 3m 24s	remaining: 18.9s
4577:	learn: 0.3852601	test: 0.4085375	best: 0.4074796 (2369)	total: 3m 24s	

4656:	learn: 0.3848092	test: 0.4085522	best: 0.4074796 (2369)	total: 3m 28s	remaining: 15.4s
4657:	learn: 0.3848012	test: 0.4085558	best: 0.4074796 (2369)	total: 3m 28s	remaining: 15.3s
4658:	learn: 0.3847915	test: 0.4085565	best: 0.4074796 (2369)	total: 3m 28s	remaining: 15.3s
4659:	learn: 0.3847881	test: 0.4085548	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15.2s
4660:	learn: 0.3847868	test: 0.4085540	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15.2s
4661:	learn: 0.3847825	test: 0.4085604	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15.2s
4662:	learn: 0.3847807	test: 0.4085593	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15.1s
4663:	learn: 0.3847782	test: 0.4085593	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15.1s
4664:	learn: 0.3847742	test: 0.4085646	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15s
4665:	learn: 0.3847697	test: 0.4085645	best: 0.4074796 (2369)	total: 3m 29s	remaining: 15s
4666:	learn: 0.3847667	test: 0.4085659	best: 0.4074796 (2369)	total: 3m 29

4746:	learn: 0.3843275	test: 0.4085802	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.4s
4747:	learn: 0.3843253	test: 0.4085752	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.4s
4748:	learn: 0.3843236	test: 0.4085767	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.3s
4749:	learn: 0.3843231	test: 0.4085767	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.3s
4750:	learn: 0.3843136	test: 0.4085877	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.2s
4751:	learn: 0.3842975	test: 0.4085842	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.2s
4752:	learn: 0.3842856	test: 0.4085794	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.2s
4753:	learn: 0.3842796	test: 0.4085799	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.1s
4754:	learn: 0.3842693	test: 0.4085783	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11.1s
4755:	learn: 0.3842651	test: 0.4085813	best: 0.4074796 (2369)	total: 3m 34s	remaining: 11s
4756:	learn: 0.3842583	test: 0.4085778	best: 0.4074796 (2369)	total: 3m 

4838:	learn: 0.3838601	test: 0.4086479	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.29s
4839:	learn: 0.3838552	test: 0.4086481	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.25s
4840:	learn: 0.3838512	test: 0.4086474	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.2s
4841:	learn: 0.3838480	test: 0.4086511	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.16s
4842:	learn: 0.3838470	test: 0.4086517	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.11s
4843:	learn: 0.3838400	test: 0.4086529	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.07s
4844:	learn: 0.3838318	test: 0.4086494	best: 0.4074796 (2369)	total: 3m 39s	remaining: 7.02s
4845:	learn: 0.3838270	test: 0.4086477	best: 0.4074796 (2369)	total: 3m 39s	remaining: 6.98s
4846:	learn: 0.3838208	test: 0.4086541	best: 0.4074796 (2369)	total: 3m 39s	remaining: 6.93s
4847:	learn: 0.3838173	test: 0.4086530	best: 0.4074796 (2369)	total: 3m 39s	remaining: 6.89s
4848:	learn: 0.3838071	test: 0.4086533	best: 0.4074796 (2369)	total: 3m

4929:	learn: 0.3833182	test: 0.4086686	best: 0.4074796 (2369)	total: 3m 43s	remaining: 3.17s
4930:	learn: 0.3833158	test: 0.4086679	best: 0.4074796 (2369)	total: 3m 43s	remaining: 3.13s
4931:	learn: 0.3833141	test: 0.4086703	best: 0.4074796 (2369)	total: 3m 43s	remaining: 3.08s
4932:	learn: 0.3833105	test: 0.4086702	best: 0.4074796 (2369)	total: 3m 43s	remaining: 3.04s
4933:	learn: 0.3833083	test: 0.4086687	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.99s
4934:	learn: 0.3833059	test: 0.4086689	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.94s
4935:	learn: 0.3832955	test: 0.4086651	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.9s
4936:	learn: 0.3832832	test: 0.4086506	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.85s
4937:	learn: 0.3832820	test: 0.4086515	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.81s
4938:	learn: 0.3832778	test: 0.4086487	best: 0.4074796 (2369)	total: 3m 43s	remaining: 2.76s
4939:	learn: 0.3832741	test: 0.4086510	best: 0.4074796 (2369)	total: 3m

## 2.2 Model cross validation
Cross validation is even better than validation.

In [34]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Now we have values of our loss functions at each boosting step averaged by 3 folds, which should provide us with a more accurate estimation of our model performance:

In [36]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.82±0.00 on step 804


In [37]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8208799722279267


## 2.3 Model applying
Apply our model to the test set. 

In [66]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 1 0 0 0 0 0 0 0 0]
[[0.63192475 0.36807525]
 [0.36730514 0.63269486]
 [0.83655699 0.16344301]
 [0.58215889 0.41784111]
 [0.79125149 0.20874851]
 [0.98437892 0.01562108]
 [0.703927   0.296073  ]
 [0.76164495 0.23835505]
 [0.5428711  0.4571289 ]
 [0.81614983 0.18385017]]


When we created the model we specified random_seed=42 parameter. By default CatBoost chooses a random value for seed.

In [18]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 0


We define parameters and Pool. The Pool stores information about the data set such as features, labels, categorical feature indices, weights and more.

In [19]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

## 3.1 Using the best model

In [20]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.8211

Best model validation accuracy: 0.8219


## 3.2 Early stopping
Early stopping saves time and improves the quality. If you have a validation set it is always better to use early stopping.

In [22]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 4min 55s, sys: 1min 18s, total: 6min 14s
Wall time: 48.7 s


In [23]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 1min 25s, sys: 23.4 s, total: 1min 48s
Wall time: 14.6 s


In [24]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Simple model tree count: 500
Simple model validation accuracy: 0.8211

Early-stopped model tree count: 155
Early-stopped model validation accuracy: 0.8204


## 3.3 Feature importance
Can be useful to understand which features that make the greatest contribution to the result. CatBoost have a get_feature_importance method.

In [27]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

f13: 19.26966240994379
f2: 9.469461241197951
f16: 8.046278024327469
f3: 6.931812113245629
f10: 5.541255734612158
f1: 4.685244139516803
f15: 4.067701032776284
f21: 3.93659232643926
f18: 3.9187243667976315
f4: 3.5888303826652375
f7: 3.2502401948432342
f8: 3.1092247520891827
f19: 3.0139757042465813
f25: 2.9658037247245206
f12: 2.6373657775987245
f9: 2.57930714958684
f11: 1.887285391110578
f24: 1.703173284588879
f27: 1.4008949113297697
f17: 1.16678399516709
f28: 1.0910009854674942
f20: 0.9739860344528134
f23: 0.9052364615596726
f26: 0.7900272173983826
f22: 0.7778379452962153
f14: 0.7625730376497267
f6: 0.7390004200041022
f5: 0.6750452333089805
f0: 0.11567600805499587
id: 0.0


## 3.4 Eval metrics

In [30]:
model = CatBoostClassifier(iterations=100, random_seed=42, logging_level='Silent').fit(train_pool)
eval_metrics = model.eval_metrics(validate_pool, [metrics.AUC()], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [29]:
print(eval_metrics['AUC'][:6])

[0.6444278842120503, 0.7005253578329491, 0.7187576324502863, 0.7288769718361316, 0.7352481227132248, 0.73954770818449]


## 4.1 Write predictions to file
Write the predictions made by the model to a csv file on the format (id, target). 

In [67]:
stacked_columns = np.column_stack((df_test["id"], predictions_probs[:, 1]))
df = pd.DataFrame(stacked_columns, columns=["id", "target"])
convert_dict={"id": int, "target" : float}
df = df.astype(convert_dict)
df.to_csv("predictions.csv", index = False)